<a href="https://colab.research.google.com/github/poralapoornachandra/Recurrent-Neural-Networks/blob/main/LSTM_Using_daily_temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Step-1 load the dataset
# Parse_dates : Used to automatoically converted a string object in to a datetime object
df = pd.read_csv("/content/daily_minimum_temps.csv", parse_dates = ["Date"],index_col = "Date")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Clean the data
df["Temp"] = pd.to_numeric(df["Temp"],errors = 'coerce') # coerce = ignore missing values
df = df.dropna()

In [ ]:
# Step - 3 Normalise the temperature values
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df["Temp"].values.reshape(-1,1))
data_scaled

In [ ]:
# Creating sequence is the most important step in RNN
# Step -4 :- Create input sequences for LSTM
def create_sequences(data, seq_length):
  X, y = [],[]
  for i in range(len(data) - seq_length):
    X.append(data[i:i+seq_length])
    y.append(data[i+seq_length])


  return np.array(X),np.array(y)

In [ ]:
seq_length = 30
X,y = create_sequences(data_scaled, seq_length)

In [ ]:
# Step -5 Train-test-split (no splitting for time series)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)

In [ ]:
# step - 6 : Build the LSTM model
model = Sequential([
    LSTM(64, activation = 'relu', input_shape = (seq_length,1)),
    Dense(1) # Output is a single value
])

In [ ]:
model.compile(optimizer='adam', loss = 'mse')

In [ ]:
model.summary()

In [ ]:
# Step - 7: Train the model
model.fit(X_train,y_train, epochs = 20, batch_size = 32)

In [ ]:
# Step - 8 :- Make predictions on the test set
y_pred_scaled = model.predict(X_test)

In [ ]:
# Clip predictions to [0,1] before inverse transform
y_pred_scaled = np.clip(y_pred_scaled, 0, 1)
y_pred = scaler.inverse_transform(y_pred_scaled)
y_test_actual = scaler.inverse_transform(y_test)

In [ ]:
# Step - 9 plot predictions
plt.figure(figsize = (12,6))
plt.plot(y_test_actual,label = "Actual Temperatures")
plt.plot(y_pred, label = "Predicted Temperatures")
plt.title("Daily Min Temperature Forecasting (LSTM)")
plt.xlabel("Time")
plt.ylabel("Temperature")
plt.legend()
plt.title("Temperature Prediction")